In [1]:
import pandas as pd
import numpy as np
import os
from PIL import Image
from tqdm import tqdm 

# PyTorch & Scikit-learn
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from sklearn.metrics import f1_score
import torch.nn.functional as F

In [ ]:
# --- 경로 설정 ---
data_dir = 'safebooru\data'
image_dir = os.path.join(data_dir, 'images')
train_csv_path = os.path.join(data_dir, 'train.csv')
val_csv_path = os.path.join(data_dir, 'val.csv')
weight_path = os.path.join(data_dir, 'tag_weights.pt')

model_dir = os.path.join(data_dir, 'model') 
model_save_path = os.path.join(model_dir, 'best_model.pth')

In [3]:
# --- 하이퍼파라미터 및 병렬 처리 설정 ---
NUM_EPOCHS = 10
BATCH_SIZE = 64 # GPU 메모리가 충분하다면 배치 크기 늘리기
LEARNING_RATE = 1e-4
FOCAL_GAMMA = 2.0
NUM_WORKERS = os.cpu_count() - 2 # 데이터 로딩에 사용할 CPU 코어 (최대 20개 정도 추천)

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ 설정 완료, 사용 장치: {device}, 데이터로더 워커: {NUM_WORKERS}")

✅ 설정 완료, 사용 장치: cpu, 데이터로더 워커: 46


In [5]:
class SafebooruDataset(Dataset):
    def __init__(self, csv_path, image_dir, transform=None):
        self.df = pd.read_csv(csv_path)
        self.image_dir = image_dir
        self.transform = transform
        self.tag_columns = [col for col in self.df.columns if col not in ['id', 'created_at', 'rating', 'score', 'sample_url', 'sample_width', 'sample_height', 'preview_url']]
        self.labels = self.df[self.tag_columns].values.astype(np.float32)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, f"{self.df.iloc[idx]['id']}.jpg")
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        labels = torch.from_numpy(self.labels[idx])
        return image, labels

print("✅ SafebooruDataset 클래스 정의 완료")

✅ SafebooruDataset 클래스 정의 완료


In [6]:
# 이미지넷의 평균과 표준편차
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

train_dataset = SafebooruDataset(csv_path=train_csv_path, image_dir=image_dir, transform=train_transform)
val_dataset = SafebooruDataset(csv_path=val_csv_path, image_dir=image_dir, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

num_tags = len(train_dataset.tag_columns)
print(f"✅ 데이터로더 생성 완료, 예측할 태그 수: {num_tags}")

✅ 데이터로더 생성 완료, 예측할 태그 수: 4031


In [7]:
# 모델 정의
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
model.fc = nn.Linear(model.fc.in_features, num_tags)
model = model.to(device)

# Focal Loss 정의
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, weight=None):
        super(FocalLoss, self).__init__()
        self.gamma = gamma; self.weight = weight
    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none', weight=self.weight)
        pt = torch.exp(-BCE_loss); focal_loss = (1 - pt)**self.gamma * BCE_loss
        return focal_loss.mean()

# 손실 함수 및 옵티마이저
tag_weights = torch.load(weight_path).to(device)
criterion = FocalLoss(gamma=FOCAL_GAMMA, weight=tag_weights)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)

print("✅ 모델, 손실 함수, 옵티마이저 정의 완료")

✅ 모델, 손실 함수, 옵티마이저 정의 완료


In [ ]:
best_f1 = 0.0

for epoch in range(NUM_EPOCHS):
    model.train()
    train_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    val_loss = 0.0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Val]"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            preds = torch.sigmoid(outputs) > 0.5
            all_preds.append(preds.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    
    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    
    print(f"\nEpoch {epoch+1}/{NUM_EPOCHS}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Macro F1: {f1:.4f}")
    
    if f1 > best_f1:
        best_f1 = f1
        torch.save(model.state_dict(), model_save_path)
        print(f"🎉 New best model saved with Macro F1: {best_f1:.4f}")

print("\n✅ 모델 학습 완료!")

Epoch 1/10 [Val]: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 71/71 [01:17<00:00,  1.10s/it]



Epoch 1/10, Train Loss: 0.0071, Val Loss: 0.0012, Macro F1: 0.0010
🎉 New best model saved with Macro F1: 0.0010


Epoch 2/10 [Val]: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 71/71 [01:13<00:00,  1.04s/it]



Epoch 2/10, Train Loss: 0.0011, Val Loss: 0.0011, Macro F1: 0.0017
🎉 New best model saved with Macro F1: 0.0017


Epoch 3/10 [Val]: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 71/71 [01:19<00:00,  1.12s/it]



Epoch 3/10, Train Loss: 0.0010, Val Loss: 0.0011, Macro F1: 0.0033
🎉 New best model saved with Macro F1: 0.0033


Epoch 4/10 [Val]: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 71/71 [01:24<00:00,  1.18s/it]



Epoch 4/10, Train Loss: 0.0009, Val Loss: 0.0010, Macro F1: 0.0090
🎉 New best model saved with Macro F1: 0.0090


Epoch 5/10 [Train]:  16%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | 89/566 [06:31<34:17,  4.31s/it]